In [1]:
import pandas as pd
import numpy as np 
import sys
sys.path.append('../../Function')
import function
from sklearn.metrics import mean_absolute_error
from keras.layers import Dense, Dropout
from keras.models import Sequential


In [2]:
df = pd.read_excel('../../../site_info_ver_5_light.xlsx')

In [17]:
dataset = df.copy()
sites = dataset.Site_Name.unique()
site_name = 'Hermon Stream (Banias)'
target = 'Israelis_Count'
target_title = 'Israelis'
dataset = dataset.loc[dataset.Site_Name==site_name]

dataset = dataset.drop(['Total','Tourists_Count'],axis=1)
dataset = dataset.drop(dataset.filter(regex='nox').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='so2').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='pm2.5').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='Ashkelon').columns, axis=1)
dataset = dataset.drop(dataset.filter(regex='Season').columns, axis=1)

print('model on site',site_name)
print('Focus on ',target_title)



dataset = function.remove_unique_one(dataset)
dataset = function.remove_high_corr(dataset, target,0.25)
dataset = function.split_date(dataset)
# dataset = function.get_weekday(dataset)
dataset = function.last_year_entries_info(dataset,target)
# dataset.set_index("Date",inplace=True)
dataset.drop("Date",axis=1,inplace=True)
dataset = dataset.astype({'day': 'category'})
dataset = dataset.astype({'month': 'category'})
dataset = dataset.astype({'year': 'category'})
dataset = dataset.drop('day', axis=1) #remove rows with nan values
dataset = dataset.drop('month', axis=1) #remove rows with nan values
dataset = dataset.drop('year', axis=1) #remove rows with nan values
print('shape of dataset',dataset.shape)
dataset.dropna(inplace=True)
print(dataset.dtypes)
print(dataset.shape)
dataset = pd.get_dummies(dataset)

print('shape of dataset',dataset.shape)
print('features :',dataset.columns)
dataset[['Israelis_Count']].describe()

model on site Hermon Stream (Banias)
Focus on  Israelis
Add Last year visitors Successfully
shape of dataset (1509, 11)
is_weekend                       int64
operations                       int64
is_jewish_holiday                int64
is_muslims_holiday               int64
pm10                           float64
Temperature                    float64
is_HeatWave                      int64
Tel_Aviv-Yafo_pm10_exceeded      int64
Beer-Sheva_pm10_exceeded         int64
Last_year_visitors             float64
Israelis_Count                   int64
dtype: object
(769, 11)
shape of dataset (769, 11)
features : Index(['is_weekend', 'operations', 'is_jewish_holiday', 'is_muslims_holiday',
       'pm10', 'Temperature', 'is_HeatWave', 'Tel_Aviv-Yafo_pm10_exceeded',
       'Beer-Sheva_pm10_exceeded', 'Last_year_visitors', 'Israelis_Count'],
      dtype='object')


,Israelis_Count
count,769.000000
mean,830.517555
std,642.993465
min,17.000000
25%,376.000000
50%,670.000000
75%,1087.000000
max,3960.000000


In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = dataset.drop(target,axis=1)
y = dataset[target]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=312148513)

X_train_scaler = MinMaxScaler()
X_test_scaler = MinMaxScaler()


X_train_scaled = X_train_scaler.fit_transform(X_train)
X_test_scaled = X_test_scaler.fit_transform(X_test)

y_train_scaled = np.log(y_train+0.001)
y_test_scaled = np.log(y_test+0.001)

dataset_train = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
dataset_test = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)



STDs

In [5]:
print("train STD " + str(np.std(y_train)))
print("test STD " + str(np.std(y_test)))

train STD 616.9170364164648
test STD 731.4666102582896


MLR Test

In [19]:
from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train_scaled,y_train)

prediction =mlr.predict(X_test_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_test.values
    },
    index=y_test.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 629.6865068536207
std 731.4666102582896


[(-356.701, 'Beer-Sheva_pm10_exceeded'),
 (-97.58, 'pm10'),
 (57.379, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (99.116, 'is_HeatWave'),
 (116.658, 'Temperature'),
 (164.917, 'operations'),
 (165.393, 'is_muslims_holiday'),
 (195.836, 'is_weekend'),
 (314.111, 'is_jewish_holiday'),
 (1117.475, 'Last_year_visitors')]

MLR Train

In [20]:
prediction =mlr.predict(X_train_scaled)

res = pd.DataFrame(
    data={
        'Prediction':prediction,
        'Actual': y_train.values
    },
    index=y_train.index
)

print('rmse',function.get_rmse(res.Prediction, res.Actual))
print('std',np.std(res.Actual))

coef = sorted( list(zip(np.round(mlr.coef_,3).T,X_train.columns)))
coef

rmse 567.6746406589097
std 616.9170364164648


[(-356.701, 'Beer-Sheva_pm10_exceeded'),
 (-97.58, 'pm10'),
 (57.379, 'Tel_Aviv-Yafo_pm10_exceeded'),
 (99.116, 'is_HeatWave'),
 (116.658, 'Temperature'),
 (164.917, 'operations'),
 (165.393, 'is_muslims_holiday'),
 (195.836, 'is_weekend'),
 (314.111, 'is_jewish_holiday'),
 (1117.475, 'Last_year_visitors')]